In [1]:
import fiftyone as fo
import fiftyone.core.labels as fol
import pandas as pd
from path_utils import path_all

## データセットの可視化


In [ ]:
# データセットの名前
DATASET_NAME = "csiro-image2biomass-train"

# 既存のデータセットがあれば削除
if fo.dataset_exists(DATASET_NAME):
    fo.delete_dataset(DATASET_NAME)

# データセットの作成
dataset = fo.Dataset(DATASET_NAME)

# CSVを読み込み、画像パスを絶対パスに変換
df = pd.read_csv(path_all.TRAIN_CSV)
df["image_path"] = df["image_path"].apply(lambda p: path_all.COMP_DIR / p)

# 1画像5ターゲットの縦長持ちを横長持ちに変換
df_pivot = df.pivot(index="image_path", columns="target_name", values="target").reset_index()

# 他のメタデータをマージ
df_meta = df[["image_path", "Sampling_Date", "State", "Species", "Pre_GSHH_NDVI", "Height_Ave_cm"]].drop_duplicates()
df_wide = pd.merge(df_pivot, df_meta, on="image_path")

# FiftyOneのサンプルを作成
samples = []
for _, row in df_wide.iterrows():
    sample = fo.Sample(filepath=row["image_path"])

    # ターゲット値をRegressionラベルとして追加
    sample["Dry_Clover_g☘️"] = fol.Regression(value=row["Dry_Clover_g"])
    sample["Dry_Dead_g🍂"] = fol.Regression(value=row["Dry_Dead_g"])
    sample["Dry_Green_g🌱"] = fol.Regression(value=row["Dry_Green_g"])
    sample["Pre_GSHH_NDVI🟢"] = fol.Regression(value=row["Pre_GSHH_NDVI"])
    sample["Height_Ave_cm📏"] = fol.Regression(value=row["Height_Ave_cm"])

    # メタデータを追加
    sample["sampling_date"] = row["Sampling_Date"]
    sample["state"] = row["State"]
    sample["species"] = row["Species"]

    samples.append(sample)

# データセットにサンプルを追加
dataset.add_samples(samples)

# ラベルの表示色を設定
dataset.app_config.color_scheme = fo.ColorScheme(
    color_by="field",
    fields=[
        {
            "path": "Dry_Clover_g☘️",
            "fieldColor": "yellowgreen",
        },
        {
            "path": "Dry_Dead_g🍂",
            "fieldColor": "brown",
        },
        {
            "path": "Dry_Green_g🌱",
            "fieldColor": "green",
        },
        {
            "path": "Pre_GSHH_NDVI🟢",
            "fieldColor": "red",
        },
        {
            "path": "Height_Ave_cm📏",
            "fieldColor": "black",
        },
    ],
)

dataset.save()  # 変更を保存

print(dataset)

# FiftyOne Appを起動
session = fo.launch_app(dataset, auto=False)
session.open_tab()


 100% |█████████████████| 357/357 [132.8ms elapsed, 0s remaining, 2.7K samples/s]   
 100% |█████████████████| 357/357 [132.8ms elapsed, 0s remaining, 2.7K samples/s]   
Name:        csiro-image2biomass-train
Media type:  image
Num samples: 357
Persistent:  False
Tags:        []
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    created_at:       fiftyone.core.fields.DateTimeField
    last_modified_at: fiftyone.core.fields.DateTimeField
    Dry_Clover_g☘️:   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Regression)
    Dry_Dead_g🍂:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Regression)
    Dry_Green_g🌱:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Regress

<IPython.core.display.Javascript object>